Tratamento dos dados do dataset "DEMO_PHQ.csv"

Realizados: 
- Verificação de respostas nulas no PHQ-9
- Tratamento de respostas `7` e/ou `9` no PHQ-9
- Cálculo da coluna `phq9`
- Cálculo da coluna `phq_grp`

Outras possibilidades
- Tratamento da coluna `INDFMINC`

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno

# Oculta warnings do notebook
import warnings
warnings.filterwarnings('ignore')

# Configurações para facilitar a visualização dos dataframes
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Carregando dados demográficos e do PHQ
dataset_path = '../data/raw/'
demo_phq_dataset_name = 'DEMO_PHQ.csv'

demo_phq = pd.read_csv(f"{dataset_path}{demo_phq_dataset_name}", sep=',')

In [ ]:
# Checando ocorrencias de respostas 7 e 9 no PHQ-9
print(f'Ocorrências de resposta 7: {(demo_phq[phq_cols] == 7.0).sum().sum()}')
print(f'Ocorrências de resposta 9: {(demo_phq[phq_cols] == 9.0).sum().sum()}')

In [ ]:
# Como estes valores significam uma ausencia de resposta, eles serão removidos para não influenciar nos cálculos
demo_phq[phq_cols] = demo_phq[phq_cols].replace([7.0, 9.0], np.nan)

In [ ]:
# Calculando score do PHQ-9
demo_phq['PHQ9'] = demo_phq[phq_cols].sum(axis=1)

In [ ]:
# Calculando grupo de sintomas de depressão
def get_phq9_group(score):
    if score < 5: return 0  # sem sintomas
    if score < 10: return 1  # sintomas leves
    if score < 15: return 2  # sintomas moderados
    if score < 19: return 3  # sintomas moderadamente severos
    return 4  # sintomas severos

demo_phq['PHQ_GRP'] = demo_phq['PHQ9'].map(get_phq9_group)

In [ ]:
# Carregando dados do PAG e HEI
pag_hei_dataset_name = 'PAG_HEI.csv'
pag_hei = pd.read_csv(f"{dataset_path}{pag_hei_dataset_name}", sep=',')

In [ ]:
# Mesclando dados de ambos datasets
df = pd.merge(demo_phq, pag_hei, on="SEQN", how="left")

# Checando dimensões após mesclagem
print("demo_phq:", demo_phq.shape)
print("pag_hei:", pag_hei.shape)
print("df mesclado:", df.shape)

In [ ]:
# Exportando o dataframe mesclado
output_path = '../data/output/'
output_file = 'treated_dataset.csv'

df.to_csv(f"{output_path}{output_file}", sep=',', encoding='utf-8', index = False)